<a href="https://colab.research.google.com/github/amitgal21/Final_Project/blob/main/calculate_bacteria_percentage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

At this stage, we are training the U-Net architecture, which deals with segmentation tasks on images with small objects. Therefore, for our project, which involves predicting and segmenting images of bacteria, the U-Net architecture is suitable. We combine it with the VGG16 model so that we can achieve high-quality prediction and segmentation results.









In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Connect to google drive for load the data set

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade Pillow
#install this pagacge for preprocess the images and for tarining our model

The code loads images and their corresponding segmentations from a directory. It then defines a U-Net architecture combined with the VGG16 model for segmenting these images. After compiling the model, it trains it on the loaded data and saves the trained model to a file. Overall, the code performs image segmentation on bacteria images using the U-Net architecture with VGG16 as a base model and saves the trained model for future use.

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout
from tensorflow.keras.applications import VGG16

def load_images_and_segmentations(data_dir):
    images = []
    segmentations = []

    for subdir in os.listdir(data_dir):
        subdir_path = os.path.join(data_dir, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                if filename.lower().endswith('.tif') and not '_segmentation.tif' in filename:
                    try:
                        img_path = os.path.join(subdir_path, filename)
                        img = Image.open(img_path)
                        img = img.resize((224, 224))
                        img = np.array(img)
                        if img.ndim == 2:
                            img = np.stack((img,) * 3, axis=-1)
                        img = img / 255.0
                        images.append(img)

                        segmentation_path = img_path.replace('.tif', '_segmentation.tif')
                        segmentation = Image.open(segmentation_path)
                        segmentation = segmentation.resize((224, 224))
                        segmentation = np.array(segmentation)
                        if segmentation.ndim == 2:
                            segmentation = np.expand_dims(segmentation, axis=-1)
                        segmentation = segmentation.astype('float32') / 255.0
                        segmentations.append(segmentation)
                        print(f"Segmentation loaded: {filename.replace('.tif', '_segmentation.tif')}")

                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(segmentations)



# Define paths
data_dir = '/content/drive/MyDrive/Part_B/Datset3/Learn'

images, segmentations = load_images_and_segmentations(data_dir)

from tensorflow.keras.layers import UpSampling2D

def vgg16_unet(input_size=(224, 224, 3), num_classes=34):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_size)
    for layer in base_model.layers:
        layer.trainable = False

    block1_pool = base_model.get_layer('block1_pool').output
    block2_pool = base_model.get_layer('block2_pool').output
    block3_pool = base_model.get_layer('block3_pool').output
    block4_pool = base_model.get_layer('block4_pool').output
    block5_pool = base_model.get_layer('block5_pool').output

    u6 = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(block5_pool)
    u6 = concatenate([u6, block4_pool])
    u6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    u6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)

    u7 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(u6)
    u7 = concatenate([u7, block3_pool])
    u7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    u7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)

    u8 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(u7)
    u8 = concatenate([u8, block2_pool])
    u8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    u8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)

    u9 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(u8)
    u9 = concatenate([u9, block1_pool])
    u9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    u9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)

    u10 = UpSampling2D((2, 2))(u9)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(u10)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model


model = vgg16_unet()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


segmentations_one_hot = tf.keras.utils.to_categorical(segmentations, num_classes=34)

model.fit(images, segmentations_one_hot, batch_size=32, epochs=10, validation_split=0.2)

model_save_path = '/content/drive/My Drive/trained_model_unet.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")



This code snippet loads a pre-trained U-Net model for image segmentation from a specified path. It then defines functions to load and prepare an image for inference, calculate the area and percentage of bacteria in the segmented image, and process images from a directory by predicting segmentation maps and analyzing them.

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

# Load the trained U-Net model for image segmentation
model_path = '/content/drive/My Drive/trained_model_unet.h5'
model = load_model(model_path)

def load_and_prepare_image(image_path):
    """
    Load and prepare an image for inference.

    Parameters:
        image_path (str): The path to the image file.

    Returns:
        numpy.ndarray: The preprocessed image ready for prediction.
    """
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize image to match model input size
    img = np.array(img)
    if img.ndim == 2:  # If the image is grayscale, convert it to RGB
        img = np.stack((img,) * 3, axis=-1)
    img = img / 255.0  # Normalize pixel values to range [0, 1]
    img = np.expand_dims(img, axis=0)  # Add a batch dimension to the image
    return img

def calculate_bacteria_area(segmentation, pixel_size=1.0):
    """
    Calculate the area and percentage of bacteria in the segmented image.

    Parameters:
        segmentation (numpy.ndarray): The segmentation map where each pixel's value represents a class.
        pixel_size (float): The size of each pixel in micrometers.

    Returns:
        tuple: A tuple containing the area of bacteria in square micrometers and the percentage of area covered by bacteria.
    """
    bacteria_pixels = np.sum(segmentation > 0)  # Count all the bacteria pixels
    total_pixels = segmentation.size  # Total pixels in the image
    area = bacteria_pixels * pixel_size * pixel_size  # Calculate the area in square micrometers
    percentage = (bacteria_pixels / total_pixels) * 100  # Calculate the percentage of area covered by bacteria
    return area, percentage

def process_images_from_directory(directory_path):
    """
    Process images from a directory by predicting segmentation maps and analyzing them.

    Parameters:
        directory_path (str): The path to the directory containing the images.
    """
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.tif'):  # Process only TIFF image files
            image_path = os.path.join(directory_path, filename)
            img = load_and_prepare_image(image_path)  # Load and prepare the image
            pred = model.predict(img)  # Predict segmentation map using the trained model
            segmentation = np.argmax(pred, axis=-1)[0]  # Extract segmentation map from predictions
            bacteria_area, bacteria_percentage = calculate_bacteria_area(segmentation)  # Calculate bacteria area and percentage
            print(f"Bacteria area in {filename}: {bacteria_area} square micrometers, which is {bacteria_percentage:.2f}% of the image")


directory_path = '/content/drive/MyDrive/Part_B/Datset3/Verify/a'
process_images_from_directory(directory_path)


The goal of this code is to train a U-Net model architecture to estimate the percentage of bacteria in an image. We aim to extract these vital data points in order to accurately assess the type of bacteria present.